In [ ]:
# default_exp models

# SecureReqNet Models

## Load SecureReqNet Model
Currently only alpha is available

In [ ]:
import os

__file__ = (os.getcwd())

In [ ]:
#export
from tensorflow.keras.models import load_model
import os
def load_securereqnet_model(model):
    '''
    Returns a pretrained version of the selected model
    
    Users can select: alpha    
    '''
    path = os.path.join(os.path.dirname(os.path.abspath(__file__)),'pretrained_models') +'/' + model + ".hdf5"
    # path = 'pretrained_models/' + model + '.hdf5'
    criticality_network_load = load_model(path) #<----- The Model'
    return criticality_network_load

In [ ]:
#export
def create_alpha(embed_size,max_len_sentences):
    '''Creates a securereqnet alpha model and compiles it'''
    #BaseLine Architecture <-------
    embeddigs_cols = embed_size
    input_sh = (max_len_sentences,embeddigs_cols,1)
    #Selecting filters? 
    #https://stackoverflow.com/questions/48243360/how-to-determine-the-filter-parameter-in-the-keras-conv2d-function
    #https://stats.stackexchange.com/questions/196646/what-is-the-significance-of-the-number-of-convolution-filters-in-a-convolutional

    N_filters = 128 # <-------- [HyperParameter] Powers of 2 Numer of Features
    K = 2 # <-------- [HyperParameter] Number of Classess

    #baseline_model = Sequential()
    gram_input = Input(shape = input_sh)

    # 1st Convolutional Layer Convolutional Layer (7-gram)
    conv_1_layer = Conv2D(filters=32, input_shape=input_sh, activation='relu', 
                          kernel_size=(7,embeddigs_cols), padding='valid')(gram_input)

    # Max Pooling 
    max_1_pooling = MaxPooling2D(pool_size=((max_len_sentences-7+1),1), strides=None, padding='valid')(conv_1_layer)

    # Fully Connected layer
    fully_connected_1_gram = Flatten()(max_1_pooling)

    fully_connected_1_gram = Reshape((32, 1, 1))(fully_connected_1_gram)

    # 2nd Convolutional Layer (5-gram)
    conv_2_layer = Conv2D(filters=64, kernel_size=(5,1), activation='relu', 
                          padding='valid')(fully_connected_1_gram)

    max_2_pooling = MaxPooling2D(pool_size=((32-5+1),1), strides=None, padding='valid')(conv_2_layer)

    # Fully Connected layer
    fully_connected_2_gram = Flatten()(max_2_pooling)

    fully_connected_2_gram = Reshape((64, 1, 1))(fully_connected_2_gram)

    # 3rd Convolutional Layer (3-gram)
    conv_3_layer =  Conv2D(filters=128, kernel_size=(3,1), activation='relu', 
                          padding='valid')(fully_connected_2_gram)

    # 4th Convolutional Layer (3-gram)
    conv_4_layer = Conv2D(filters=128, kernel_size=(3,1), activation='relu', 
                         padding='valid')(conv_3_layer)

    # 5th Convolutional Layer (3-gram)
    conv_5_layer = Conv2D(filters=64, kernel_size=(3,1), activation='relu', 
                         padding='valid')(conv_4_layer)

    # Max Pooling
    max_5_pooling = MaxPooling2D(pool_size=(58,1), strides=None, padding='valid')(conv_5_layer)

    # Fully Connected layer
    fully_connected = Flatten()(max_5_pooling)

    # 1st Fully Connected Layer
    deep_dense_1_layer = Dense(32, activation='relu')(fully_connected)
    deep_dense_1_layer = Dropout(0.2)(deep_dense_1_layer) # <-------- [HyperParameter]

    # 2nd Fully Connected Layer
    deep_dense_2_layer = Dense(32, activation='relu')(deep_dense_1_layer)
    deep_dense_2_layer = Dropout(0.2)(deep_dense_2_layer) # <-------- [HyperParameter]

    # 3rd Fully Connected Layer
    deep_dense_3_layer = Dense(16, activation='relu')(deep_dense_2_layer)
    deep_dense_3_layer = Dropout(0.2)(deep_dense_3_layer) # <-------- [HyperParameter]

    predictions = Dense(K, activation='softmax')(deep_dense_3_layer)

    #Criticality Model
    criticality_network = Model(inputs=[gram_input],outputs=[predictions]) 
    print(criticality_network.summary())
    
    #Seting up the Model
    criticality_network.compile(optimizer='adam',loss='binary_crossentropy',
                                  metrics=['accuracy'])
    
    return criticality_network


In [ ]:
#export
def fit_model(corpora_train_x,train_y,criticality_network,save_model=True,save_history=True,path=''):
    '''
    Fits a securereqnet model to the data
    
    Returns model, history
    
    @param save_model (bool): Saves the model to the input path
    
    @param save_history (bool): Saves the history to the path
    
    @param path (string): Path to where the model and/or history will be saved to
    '''
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
    mc = ModelCheckpoint(filepath, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    callbacks_list = [es,mc]

    #Model Fitting
    history = criticality_network.fit(
            x = corpora_train_x, 
            y = target_train_y,
            #batch_size=64,
            epochs=2000, #5 <------ Hyperparameter
            validation_split = 0.2,
            callbacks=callbacks_list
    )
    
    #CheckPoints
    #csv_logger = CSVLogger(system+'_training.log')
    # filepath changed from: "alex-adapted-res-003/best_model.hdf5" for testing
    # The folder alex-adapted-res-003 doesn't exist yet in the repository. RC created 08_test in the root folder 
    # manually
    if(save_model):
        filepath = path+"/best_model.hdf5"
        criticality_network.save(filepath)
    
    # filepath changed from: 'alex-adapted-res-003/history_training.csv' for testing
    #Saving Training History
    if(save_history):
        df_history = pd.DataFrame.from_dict(history.history)
        df_history.to_csv(path+'/history_training.csv', encoding='utf-8',index=False)
        df_history.head()
    
    return criticality_network, history

In [ ]:
from securereqnet.preprocessing import *
path = "../securereqnet/securereqnet_data/augmented_dataset/"
corpora_train,corpora_test,max_len_sentences,embed_size = get_training_corpora(path,True)

[nltk_data] Downloading package stopwords to /home/roger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


UnboundLocalError: local variable 'contents' referenced before assignment

In [ ]:
corpora_train_x, corpora_test_x, target_train_y, target_test_y = process_corpora(corpora_train,corpora_test,save_file=True,path="../alpha",name="alpha-1")

In [ ]:
model = load_securereqnet_model("alpha")
model.evaluate(corpora_test_x, target_test_y, verbose=1)

NameError: name 'corpora_test_x' is not defined

In [ ]:
model2, history = fit_model(corpora_train_x,train_y,criticality_network,save_model=True,save_history=True,path='')
model2.evaluate(corpora_test_x, target_test_y, verbose=1)


NameError: name 'fit_model' is not defined

In [ ]:
from securereqnet.preprocessing import vectorize_sentences
model.predict(vectorize_sentences(["yes"]))

array([[0.1026049, 0.8973951]], dtype=float32)